## Demo

### Connect KL520

In [1]:
import cv2
import kp

In [2]:
device_descriptors = kp.core.scan_devices()

if 0 < device_descriptors.device_descriptor_number:
    usb_port_id = device_descriptors.device_descriptor_list[0].usb_port_id
    print(usb_port_id)
else:
    print('Error: no Kneron device connect.')
    exit(0)

22


In [3]:
device_group = kp.core.connect_devices(usb_port_ids=[22])

In [6]:
kp.core.set_timeout(device_group=device_group,
                    milliseconds=5000)

### Load firmware, model, and test image

In [7]:
SCPU_FW_PATH = '../../res/firmware/KL520/fw_scpu.bin'
NCPU_FW_PATH = '../../res/firmware/KL520/fw_ncpu.bin'
kp.core.load_firmware_from_file(device_group=device_group,
                                scpu_fw_path=SCPU_FW_PATH,
                                ncpu_fw_path=NCPU_FW_PATH)

In [8]:
MODEL_FILE_PATH = './photo_scorer_520.nef'
model_nef_descriptor = kp.core.load_model_from_file(device_group=device_group,
                                                    file_path=MODEL_FILE_PATH)

### pre-processing

In [9]:
IMAGE_FILE_PATH = './test.jpg'

img = cv2.imread(filename=IMAGE_FILE_PATH)
img_bgr565 = cv2.cvtColor(src=img, code=cv2.COLOR_BGR2BGR565)

In [10]:
generic_inference_input_descriptor = kp.GenericImageInferenceDescriptor(
    model_id=model_nef_descriptor.models[0].id,
    inference_number=0,
    input_node_image_list=[
        kp.GenericInputNodeImage(
            image=img_bgr565,
            image_format=kp.ImageFormat.KP_IMAGE_FORMAT_RGB565,
            resize_mode=kp.ResizeMode.KP_RESIZE_ENABLE,
            padding_mode=kp.PaddingMode.KP_PADDING_CORNER,
            normalize_mode=kp.NormalizeMode.KP_NORMALIZE_KNERON
        )
    ]
)

In [11]:
kp.inference.generic_image_inference_send(device_group=device_group,
                                          generic_inference_input_descriptor=generic_inference_input_descriptor)
generic_raw_result = kp.inference.generic_image_inference_receive(device_group=device_group)
print(generic_raw_result)

{
    "header": {
        "inference_number": 0,
        "crop_number": 0,
        "num_output_node": 1,
        "product_id": 256,
        "num_hw_pre_proc_info": 1,
        "hw_pre_proc_info_list": {
            "0": {
                "img_width": 1200,
                "img_height": 800,
                "resized_img_width": 224,
                "resized_img_height": 149,
                "pad_top": 0,
                "pad_bottom": 75,
                "pad_left": 0,
                "pad_right": 0,
                "model_input_width": 224,
                "model_input_height": 224,
                "crop_area": {
                    "crop_box_index": 0,
                    "x": 0,
                    "y": 0,
                    "width": 0,
                    "height": 0
                }
            }
        }
    },
    "raw_result": {
        "buffer_size": 388
    }
}


In [13]:
inf_node_output_list = []

for node_idx in range(generic_raw_result.header.num_output_node):
    inference_float_node_output = kp.inference.generic_inference_retrieve_float_node(node_idx=node_idx,
                                                                                     generic_raw_result=generic_raw_result,
                                                                                     channels_ordering=kp.ChannelOrdering.KP_CHANNEL_ORDERING_CHW)
    inf_node_output_list.append(inference_float_node_output)

print(inf_node_output_list)

[{
    "width": 1,
    "height": 1,
    "channel": 1,
    "channels_ordering": "ChannelOrdering.KP_CHANNEL_ORDERING_CHW",
    "num_data": 1,
    "ndarray": [
        "[[[[0.71443015]]]]"
    ]
}]


### post-processing
Kneron PLUS python version doesn't support on-NPU post-processing, so this step is demonstrated here

In [23]:
import numpy as np

nd_array = inf_node_output_list[0].ndarray
number = float(nd_array.flatten()[0])

print("The score of this photo is", number)

The score of this photo is 0.7144301533699036
